In [160]:
import pandas as pd
import numpy as np

In [161]:
from tensorflow import keras
import tensorflow as tf

In [162]:
from sklearn.preprocessing import LabelEncoder

In [163]:
from sklearn.model_selection import KFold

In [164]:
from IPython.display import clear_output

In [165]:
from matplotlib import pyplot as plt
import os, sys
sys.path.append(os.path.abspath("../../plots/"))
from plotter import plot_marginals

In [166]:
from itertools import product

In [167]:
from functools import reduce
import operator
def prod(iterable):
    return reduce(operator.mul, iterable, 1)
# from math import prod

In [168]:
from io import StringIO

# Data preparation

Data can be found [here](http://mlr.cs.umass.edu/ml/machine-learning-databases/census-income/).

In [169]:
data = pd.read_csv(
#     "/cluster/scratch/bramva/deep-learning-course/data/census-income/census-income.data",
    "data/census-income.data",
    names=[
        "age",
        "workclass",
        "fnlwgt",
        "education",
        "education-num",
        "marital-status",
        "occupation",
        "relationship",
        "race",
        "sex",
        "capital-gain",
        "capital-loss",
        "hours-per-week",
        "native-country",
        "income",
    ],
)

In [170]:
categoricals = [
    "workclass",
    "education",
    "marital-status",
    "occupation",
    "relationship",
    "race",
    "sex",
    "native-country",
    "income",
]

In [171]:
data.describe(include="all")

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
count,32561.000000,32561,3.256100e+04,32561,32561.000000,32561,32561,32561,32561,32561,32561.000000,32561.000000,32561.000000,32561,32561
unique,NaN,9,NaN,16,NaN,7,15,6,5,2,NaN,NaN,NaN,42,2
top,NaN,Private,NaN,HS-grad,NaN,Married-civ-spouse,Prof-specialty,Husband,White,Male,NaN,NaN,NaN,United-States,<=50K
freq,NaN,22696,NaN,10501,NaN,14976,4140,13193,27816,21790,NaN,NaN,NaN,29170,24720
mean,38.581647,NaN,1.897784e+05,NaN,10.080679,NaN,NaN,NaN,NaN,NaN,1077.648844,87.303830,40.437456,NaN,NaN
std,13.640433,NaN,1.055500e+05,NaN,2.572720,NaN,NaN,NaN,NaN,NaN,7385.292085,402.960219,12.347429,NaN,NaN
min,17.000000,NaN,1.228500e+04,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,1.000000,NaN,NaN
25%,28.000000,NaN,1.178270e+05,NaN,9.000000,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,40.000000,NaN,NaN
50%,37.000000,NaN,1.783560e+05,NaN,10.000000,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,40.000000,NaN,NaN
75%,48.000000,NaN,2.370510e+05,NaN,12.000000,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,45.000000,NaN,NaN


In [172]:
data.iloc[0, :]

age                           39
workclass              State-gov
fnlwgt                     77516
education              Bachelors
education-num                 13
marital-status     Never-married
occupation          Adm-clerical
relationship       Not-in-family
race                       White
sex                         Male
capital-gain                2174
capital-loss                   0
hours-per-week                40
native-country     United-States
income                     <=50K
Name: 0, dtype: object

In [173]:
for categorical in categoricals:
    data[categorical] = LabelEncoder().fit_transform(data[categorical])
    values = data[categorical].unique()
    np.random.shuffle(values)
    data[categorical] = values[data[categorical]]

In [174]:
means = data.mean()

In [175]:
# data -= means

In [176]:
stds = data.std()

In [177]:
# data /= stds

In [178]:
dataset = data.values

In [179]:
ordering = np.arange(dataset.shape[0])
np.random.shuffle(ordering)
dataset = dataset[ordering]

In [180]:
cutoff = round(dataset.shape[0] * .1)

In [181]:
train, labels = dataset[:, :-1], dataset[:, -1]

In [182]:
labels

array([1, 1, 1, ..., 1, 1, 1])

In [183]:
dataset.shape

(32561, 15)

# Utilities

In [184]:
class PlotLosses(keras.callbacks.Callback):
    def __init__(
        self,
        metrics=None,
        validation=True,
    ):
        self.metrics = ["loss"]
        if metrics:
            self.metrics += metrics
        if validation:
            for i in reversed(range(len(self.metrics))):
                self.metrics.insert(i + 1, "val_" + self.metrics[i])
        self.hist_metrics = [list() for metric in self.metrics]
        
    def on_train_begin(self, logs={}):
        self.logs = []

    def on_epoch_end(self, epoch, logs={}):
        self.logs.append(logs)
        for metric, hist_metric in zip(self.metrics, self.hist_metrics):
            hist_metric.append(logs.get(metric))

        clear_output(wait=True)
        
        self.fig = plt.figure(figsize=(15, 7))
        for metric, hist_metric in zip(self.metrics, self.hist_metrics):
            plt.plot(range(len(self.hist_metrics[0])), hist_metric, label=metric)
        plt.legend()
        plt.show()

    def __len__(self):
        return len(self.hist_metrics[0])

# Model

In [185]:
def build_model(
    before_trainable_layers=[],
    random_layers=[],
    after_trainable_layers=[],
    regularization=1e-6,
):
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape=(train.shape[1])))
    
    for i, layer_size in enumerate(before_trainable_layers):
        model.add(keras.layers.Dense(
            layer_size,
            activation="relu",
            kernel_regularizer=keras.regularizers.l2(regularization),
        ))
        
    for random_layer_size in random_layers:
        model.add(keras.layers.Dense(
            random_layer_size,
            activation="tanh",
            trainable=False,
        ))
    
    previous_size = random_layers[-1] if random_layers else before_trainable_layers[-1] if before_trainable_layers else train.shape[1]
    for i, layer_size in enumerate(after_trainable_layers):
        model.add(keras.layers.Dense(
            layer_size,
            activation="relu",
            kernel_regularizer=keras.regularizers.l2(regularization),
        ))
        
    model.add(keras.layers.Dense(2, activation='softmax'))
    return model

In [186]:
folds = 10

In [187]:
sizes = [10, 100]
max_random_depth = 2

before_options = [[], [100]]
random_options = [[size] * i for size in sizes for i in range(max_random_depth + 1)]
after_options = [[], [10]]

In [188]:
model_configurations = np.array(list(product(before_options, random_options, after_options)))
model_configurations = model_configurations.reshape(
    len(before_options),
    len(sizes),
    max_random_depth + 1,
    len(after_options),
    len((before_options, random_options, after_options)),
)
model_configurations.shape, prod(model_configurations.shape), prod(model_configurations.shape) * folds

((2, 2, 3, 2, 3), 72, 720)

In [189]:
results = np.zeros(model_configurations.shape[:-1] + (folds,))

In [190]:
for i, model_type in enumerate(model_configurations.reshape(-1, 3)):
    for j, (train_ids, test_ids) in enumerate(KFold(n_splits=folds, shuffle=True).split(np.arange(dataset.shape[0]))):
        half_validate = round(len(test_ids) / 2)
        
        model = build_model(*model_type)
        model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
        model.fit(
            train[train_ids], labels[train_ids],
            64,
            epochs=100,
            shuffle=True,
            validation_data=(
                train[test_ids][:half_validate], labels[test_ids][:half_validate],
            ),
            callbacks=[
                keras.callbacks.EarlyStopping(
                    patience=5,
                    restore_best_weights=True,
                )
#                 PlotLosses(),
            ],
        )
        
        score = model.evaluate(
            train[test_ids][round(len(test_ids) / 2):], labels[test_ids][round(len(test_ids) / 2):],
            verbose=0,
        )
        results.reshape(-1, folds)[i, j] = score[1]

Train on 29304 samples, validate on 1628 samples
Epoch 1/100
29304/29304 [==============================] - 1s 39us/sample - loss: 17848.8264 - accuracy: 0.4751 - val_loss: 13.6767 - val_accuracy: 0.7703
Epoch 2/100
29304/29304 [==============================] - 1s 29us/sample - loss: 11.4582 - accuracy: 0.7256 - val_loss: 11.1777 - val_accuracy: 0.5190
Epoch 3/100
29304/29304 [==============================] - 1s 27us/sample - loss: 10.4494 - accuracy: 0.7168 - val_loss: 25.8462 - val_accuracy: 0.7518
Epoch 4/100
29304/29304 [==============================] - 1s 27us/sample - loss: 13.9463 - accuracy: 0.7050 - val_loss: 12.4789 - val_accuracy: 0.7697
Epoch 5/100
29304/29304 [==============================] - 1s 26us/sample - loss: 17.5554 - accuracy: 0.6981 - val_loss: 24.8273 - val_accuracy: 0.7783
Epoch 6/100
29304/29304 [==============================] - 1s 27us/sample - loss: 13.8884 - accuracy: 0.6963 - val_loss: 47.1918 - val_accuracy: 0.7611
Epoch 7/100
29304/29304 [===========

29305/29305 [==============================] - 1s 22us/sample - loss: 18.3774 - accuracy: 0.6817 - val_loss: 6.4942 - val_accuracy: 0.7862
Epoch 8/100
29305/29305 [==============================] - 1s 22us/sample - loss: 15.9430 - accuracy: 0.6873 - val_loss: 7.4317 - val_accuracy: 0.7905
Epoch 9/100
29305/29305 [==============================] - 1s 21us/sample - loss: 18.3943 - accuracy: 0.6867 - val_loss: 10.1707 - val_accuracy: 0.2783
Epoch 10/100
29305/29305 [==============================] - 1s 22us/sample - loss: 14.8774 - accuracy: 0.6862 - val_loss: 3.5733 - val_accuracy: 0.7346
Epoch 11/100
29305/29305 [==============================] - 1s 22us/sample - loss: 9.2493 - accuracy: 0.6967 - val_loss: 9.8643 - val_accuracy: 0.7967
Epoch 12/100
29305/29305 [==============================] - 1s 26us/sample - loss: 15.9863 - accuracy: 0.6904 - val_loss: 7.5984 - val_accuracy: 0.7819
Epoch 13/100
29305/29305 [==============================] - 1s 27us/sample - loss: 12.9053 - accuracy: 

29304/29304 [==============================] - 1s 32us/sample - loss: 33.7303 - accuracy: 0.6780 - val_loss: 19.1238 - val_accuracy: 0.7869
Epoch 5/100
29304/29304 [==============================] - 1s 26us/sample - loss: 30.1597 - accuracy: 0.6800 - val_loss: 37.3208 - val_accuracy: 0.7733
Epoch 6/100
29304/29304 [==============================] - 1s 26us/sample - loss: 41.1225 - accuracy: 0.6827 - val_loss: 29.4315 - val_accuracy: 0.7813
Epoch 7/100
29304/29304 [==============================] - 1s 29us/sample - loss: 30.1093 - accuracy: 0.6805 - val_loss: 44.3589 - val_accuracy: 0.7715
Train on 29305 samples, validate on 1628 samples
Epoch 1/100
29305/29305 [==============================] - 1s 43us/sample - loss: 58.8820 - accuracy: 0.6677 - val_loss: 92.6131 - val_accuracy: 0.7875
Epoch 2/100
29305/29305 [==============================] - 1s 31us/sample - loss: 30.8860 - accuracy: 0.6820 - val_loss: 5.7185 - val_accuracy: 0.4324
Epoch 3/100
29305/29305 [===========================

29305/29305 [==============================] - 1s 40us/sample - loss: 41.9860 - accuracy: 0.6818 - val_loss: 56.0516 - val_accuracy: 0.7899
Epoch 8/100
29305/29305 [==============================] - 1s 50us/sample - loss: 51.0130 - accuracy: 0.6822 - val_loss: 134.0237 - val_accuracy: 0.7869
Epoch 9/100
29305/29305 [==============================] - 1s 39us/sample - loss: 47.9907 - accuracy: 0.6875 - val_loss: 21.5541 - val_accuracy: 0.7961
Epoch 10/100
29305/29305 [==============================] - 1s 34us/sample - loss: 39.7725 - accuracy: 0.6884 - val_loss: 28.3427 - val_accuracy: 0.7924
Epoch 11/100
29305/29305 [==============================] - 1s 29us/sample - loss: 35.4123 - accuracy: 0.6845 - val_loss: 64.7004 - val_accuracy: 0.7924
Train on 29305 samples, validate on 1628 samples
Epoch 1/100
29305/29305 [==============================] - 1s 43us/sample - loss: 796.4965 - accuracy: 0.6789 - val_loss: 18.2607 - val_accuracy: 0.7666
Epoch 2/100
29305/29305 [======================

29305/29305 [==============================] - 1s 29us/sample - loss: 54.6101 - accuracy: 0.6854 - val_loss: 16.1154 - val_accuracy: 0.8047
Epoch 9/100
29305/29305 [==============================] - 1s 40us/sample - loss: 45.9435 - accuracy: 0.6852 - val_loss: 45.7437 - val_accuracy: 0.7862
Epoch 10/100
29305/29305 [==============================] - 1s 29us/sample - loss: 44.9740 - accuracy: 0.6876 - val_loss: 19.3077 - val_accuracy: 0.8022
Train on 29305 samples, validate on 1628 samples
Epoch 1/100
29305/29305 [==============================] - 1s 39us/sample - loss: 2887.5321 - accuracy: 0.5964 - val_loss: 33.1912 - val_accuracy: 0.7936
Epoch 2/100
29305/29305 [==============================] - 1s 25us/sample - loss: 48.2455 - accuracy: 0.6849 - val_loss: 29.1125 - val_accuracy: 0.8010
Epoch 3/100
29305/29305 [==============================] - 1s 25us/sample - loss: 33.0583 - accuracy: 0.6885 - val_loss: 72.1822 - val_accuracy: 0.7832
Epoch 4/100
29305/29305 [=======================

...

Epoch 9/100
29305/29305 [==============================] - 3s 97us/sample - loss: 0.5403 - accuracy: 0.7682 - val_loss: 0.5217 - val_accuracy: 0.7875
Epoch 10/100
29305/29305 [==============================] - 2s 61us/sample - loss: 0.5413 - accuracy: 0.7669 - val_loss: 0.5180 - val_accuracy: 0.7869
Epoch 11/100
29305/29305 [==============================] - 3s 93us/sample - loss: 0.5456 - accuracy: 0.7647 - val_loss: 0.5185 - val_accuracy: 0.7856
Train on 29305 samples, validate on 1628 samples
Epoch 1/100
29305/29305 [==============================] - 15s 522us/sample - loss: 0.5504 - accuracy: 0.7635 - val_loss: 0.5448 - val_accuracy: 0.76900
Epoch 2/100
29305/29305 [==============================] - 6s 201us/sample - loss: 0.5463 - accuracy: 0.7655 - val_loss: 0.5392 - val_accuracy: 0.7690
Epoch 3/100
29305/29305 [==============================] - 4s 152us/sample - loss: 0.5450 - accuracy: 0.7637 - val_loss: 0.5425 - val_accuracy: 0.7672
Epoch 4/100
29305/29305 [===================

29305/29305 [==============================] - 2s 59us/sample - loss: 0.5326 - accuracy: 0.7750 - val_loss: 0.5358 - val_accuracy: 0.7844
Epoch 5/100
29305/29305 [==============================] - 2s 60us/sample - loss: 0.5238 - accuracy: 0.7839 - val_loss: 0.5249 - val_accuracy: 0.7776
Epoch 6/100
29305/29305 [==============================] - 2s 57us/sample - loss: 0.5282 - accuracy: 0.7747 - val_loss: 0.5111 - val_accuracy: 0.7752
Epoch 7/100
29305/29305 [==============================] - 2s 58us/sample - loss: 0.5143 - accuracy: 0.7856 - val_loss: 0.5054 - val_accuracy: 0.7961
Epoch 8/100
29305/29305 [==============================] - 2s 56us/sample - loss: 0.5297 - accuracy: 0.7771 - val_loss: 0.5215 - val_accuracy: 0.7752
Epoch 9/100
29305/29305 [==============================] - 2s 60us/sample - loss: 0.5258 - accuracy: 0.7776 - val_loss: 0.5608 - val_accuracy: 0.7690
Epoch 10/100
29305/29305 [==============================] - 2s 63us/sample - loss: 0.5357 - accuracy: 0.7700 - v

Epoch 7/100
29305/29305 [==============================] - 3s 96us/sample - loss: 0.5419 - accuracy: 0.7681 - val_loss: 0.5398 - val_accuracy: 0.7697
Epoch 8/100
29305/29305 [==============================] - 2s 77us/sample - loss: 0.5450 - accuracy: 0.7668 - val_loss: 0.5408 - val_accuracy: 0.7703
Epoch 9/100
29305/29305 [==============================] - 2s 58us/sample - loss: 0.5331 - accuracy: 0.7691 - val_loss: 0.5205 - val_accuracy: 0.7746
Epoch 10/100
29305/29305 [==============================] - 3s 92us/sample - loss: 0.5369 - accuracy: 0.7712 - val_loss: 0.5370 - val_accuracy: 0.7709
Epoch 11/100
29305/29305 [==============================] - 3s 104us/sample - loss: 0.5351 - accuracy: 0.7697 - val_loss: 0.5346 - val_accuracy: 0.7733
Epoch 12/100
29305/29305 [==============================] - 3s 92us/sample - loss: 0.5301 - accuracy: 0.7726 - val_loss: 0.5358 - val_accuracy: 0.7709
Epoch 13/100
29305/29305 [==============================] - 3s 99us/sample - loss: 0.5327 - accu

29304/29304 [==============================] - 2s 63us/sample - loss: 0.5423 - accuracy: 0.7666 - val_loss: 0.5672 - val_accuracy: 0.7439
Epoch 7/100
29304/29304 [==============================] - 2s 58us/sample - loss: 0.5406 - accuracy: 0.7682 - val_loss: 0.5686 - val_accuracy: 0.7463
Epoch 8/100
29304/29304 [==============================] - 2s 56us/sample - loss: 0.5436 - accuracy: 0.7664 - val_loss: 0.5655 - val_accuracy: 0.7463
Epoch 9/100
29304/29304 [==============================] - 2s 58us/sample - loss: 0.5433 - accuracy: 0.7664 - val_loss: 0.5653 - val_accuracy: 0.7463
Epoch 10/100
29304/29304 [==============================] - 2s 57us/sample - loss: 0.5341 - accuracy: 0.7733 - val_loss: 0.5584 - val_accuracy: 0.7518
Train on 29305 samples, validate on 1628 samples
Epoch 1/100
29305/29305 [==============================] - 3s 93us/sample - loss: 0.5732 - accuracy: 0.7501 - val_loss: 0.5188 - val_accuracy: 0.7875
Epoch 2/100
29305/29305 [==============================] - 2s 

29305/29305 [==============================] - 2s 69us/sample - loss: 0.5349 - accuracy: 0.7728 - val_loss: 0.5278 - val_accuracy: 0.7807
Epoch 7/100
29305/29305 [==============================] - 2s 74us/sample - loss: 0.5369 - accuracy: 0.7741 - val_loss: 0.5254 - val_accuracy: 0.7795
Train on 29305 samples, validate on 1628 samples
Epoch 1/100
29305/29305 [==============================] - 3s 113us/sample - loss: 0.5382 - accuracy: 0.7690 - val_loss: 0.5331 - val_accuracy: 0.7746
Epoch 2/100
29305/29305 [==============================] - 3s 89us/sample - loss: 0.5260 - accuracy: 0.7768 - val_loss: 0.5327 - val_accuracy: 0.7752
Epoch 3/100
29305/29305 [==============================] - 3s 86us/sample - loss: 0.5398 - accuracy: 0.7657 - val_loss: 0.5294 - val_accuracy: 0.7746
Epoch 4/100
29305/29305 [==============================] - 2s 59us/sample - loss: 0.5403 - accuracy: 0.7668 - val_loss: 0.5368 - val_accuracy: 0.7709
Epoch 5/100
29305/29305 [==============================] - 2s 

In [191]:
results.shape

(2, 2, 3, 2, 10)

In [192]:
for model_type, result_list in zip(model_configurations.reshape(-1, 3), results.reshape(-1, folds)):
    print(*model_type)
#     print(result_list)
    print("\n", "score:", np.mean(result_list), "\t", np.std(result_list), "\n\n")

[] [] []

 score: 0.7364400446414947 	 0.057947825620982094 


[] [] [10]

 score: 0.71530821621418 	 0.10936694409207603 


[] [10] []

 score: 0.7640192866325378 	 0.010948735444381371 


[] [10] [10]

 score: 0.7672739028930664 	 0.013065706627399845 


[] [10, 10] []

 score: 0.768686980009079 	 0.0202121555340608 


[] [10, 10] [10]

 score: 0.7675207436084748 	 0.013214974392625005 


[] [] []

 score: 0.6669680178165436 	 0.11115467014377993 


[] [] [10]

 score: 0.7310331553220749 	 0.13542186683272142 


[] [100] []

 score: 0.7865610718727112 	 0.016754408284159944 


[] [100] [10]

 score: 0.7927030503749848 	 0.011699123149039129 


[] [100, 100] []

 score: 0.7915359735488892 	 0.01076648601776583 


[] [100, 100] [10]

 score: 0.7858246445655823 	 0.009665232487116761 


[100] [] []

 score: 0.7072006374597549 	 0.1384969545264707 


[100] [] [10]

 score: 0.7046885848045349 	 0.11379892695052635 


[100] [10] []

 score: 0.7594126999378205 	 0.0100358809811599 


[100] 

In [193]:
with open("output/resultClassication.output", 'w+') as file:
    file.writelines([str(results.shape).lstrip("(").rstrip(")") + "\n"])
    np.savetxt(file, results.reshape(-1, folds))